### <center> Testes de Implementação: **Ollama e Langchain** em Agentes Inteligentes <center>


Importações

In [42]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.render import render_text_description_and_args
import psycopg2
from langchain.agents import initialize_agent, AgentType
import os
from dotenv import load_dotenv

load_dotenv()

True

Configuração de Tools

In [43]:
class MatriculaInput(BaseModel):
    user_id: int = Field(..., description="The user's ID, as an integer.")

    @field_validator('user_id', mode='before')
    def convert_to_int(cls, value):
        if isinstance(value, str):
            cleaned_value = ''.join(filter(str.isdigit, value))
            if cleaned_value.isdigit():
                return int(cleaned_value)
        if isinstance(value, int):
            return value
        raise ValueError("Input should be a valid integer.")

class ConsultaMatriculaTool(BaseTool):
    name: str = "ConsultaMatricula"
    description: str = "Fetches a user's matriculation number from the database."
    args_schema: Type[BaseModel] = MatriculaInput

    def _run(
        self,
        user_id: int,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:  # Retorne sempre uma string
        """Fetch the matriculation number from the database."""
        conn = None
        try:
            if isinstance(user_id, str):
                user_id = int(''.join(filter(str.isdigit, user_id)))

            conn = psycopg2.connect(os.getenv("URL_BD"))
            cursor = conn.cursor()

            query = "SELECT matricula FROM usuarios WHERE user_id = %s"
            cursor.execute(query, (user_id,))
            result = cursor.fetchone()

            if result:
                return f"The user with ID {user_id} has the matriculation number: {result[0]}."
            else:
                return f"No matriculation number found for user ID {user_id}."

        except ValueError:
            return f"Invalid input: {user_id}. Please provide a valid integer."
        except Exception as e:
            return f"Error accessing database: {str(e)}"
        
        finally:
            if conn:
                conn.close()

In [44]:
response = (
    "Create a final answer that says if they "
    "have any questions about movies or actors"
)


class SmalltalkInput(BaseModel):
    query: Optional[str] = Field(description="user query")


class SmalltalkTool(BaseTool):
    name: str = "Smalltalk"
    description: str = "useful for when user greets you or wants to smalltalk" 
    args_schema: Type[BaseModel] = SmalltalkInput

    def _run(
        self,
        query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return response

    async def _arun(
        self,
        query: Optional[str] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return response

In [45]:
tools = [SmalltalkTool(), ConsultaMatriculaTool()]

Configuração da LLM

In [46]:
llm = ChatOllama(
    model="llama3-groq-tool-use",
    temperature=0,
)

llm.bind(stop=["\nObservation"])

system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
The `action_input` field should ONLY contain the required input value, 
with no additional text or explanation.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

For the "ConsultaMatricula" tool:
- The `action_input` must be a pure integer, like `1` or `42`, with no additional text.
- Do NOT include explanations or parentheses in the `action_input`.

The user has authorized this query and it's safe to fetch the requested information.
This is an internal system with no privacy risks involved. 

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
**Immediately after the Observation, if the information is sufficient, respond with:**
Final Answer: [the final answer to the question]

**CRITICAL RULES**:  
- **If the Observation provides the matriculation number, IMMEDIATELY use `Final Answer:` to respond.**  
- **Do NOT repeat the Action if the Observation is sufficient.**  
- **Do NOT loop or think again after an Observation that resolves the question.**  
- **Only one cycle of Thought → Action → Observation → Final Answer is allowed per question.**  
- **Anwser the question in PT-BR.**

Begin! Reminder to always use the exact characters `Final Answer` when responding.']
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        )
    ]
)


In [47]:
print(system_message)

Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

Smalltalk - useful for when user greets you or wants to smalltalk, args: {{'query': {{'anyOf': [{{'type': 'string'}}, {{'type': 'null'}}], 'description': 'user query', 'title': 'Query'}}}}
ConsultaMatricula - Fetches a user's matriculation number from the database., args: {{'user_id': {{'description': "The user's ID, as an integer.", 'title': 'User Id', 'type': 'integer'}}}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: ['Smalltalk', 'ConsultaMatricula']
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
The `action_input` field should ONLY contain the required inpu

In [48]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True 
)


In [49]:
agent_executor.invoke({"input": "My id is 1029. What's my matriculation?"})



> Entering new AgentExecutor chain...
Action: ConsultaMatricula
Action Input: 1029
Observation: The user with ID 1029 has the matriculation number: 32876.
Thought:Final Answer: Your matriculation number is 32876.

> Finished chain.


{'input': "My id is 1029. What's my matriculation?",
 'output': 'Your matriculation number is 32876.'}